<a href="https://colab.research.google.com/github/AsyncHuman/portfolio/blob/main/DA_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("""Проект №1: E-commerce & Маркетинг
Тема: Юнит-экономика и когортный анализ (LTV & Retention)""")

Проект №1: E-commerce & Маркетинг
Тема: Юнит-экономика и когортный анализ (LTV & Retention)


In [2]:
import os

os.environ['KAGGLE_KEY'] = "KGAT_3a58ce8d1a1029a4893efdfed0952b1c"
os.environ['KAGGLE_USERNAME'] = "AsyncHuman"

!pip install -q kaggle
!kaggle datasets download -d olistbr/brazilian-ecommerce
!unzip -o brazilian-ecommerce.zip

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
  0% 0.00/42.6M [00:00<?, ?B/s]
100% 42.6M/42.6M [00:00<00:00, 1.35GB/s]
Archive:  brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


In [3]:
import pandas as pd

orders = pd.read_csv('olist_orders_dataset.csv')
customers = pd.read_csv('olist_customers_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')

df = pd.merge(orders, customers, on='customer_id')

df = pd.merge(df, payments, on='order_id')

print(f"Размер таблицы: {df.shape}")
df.head()

Размер таблицы: (103886, 16)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,payment_sequential,payment_type,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1,credit_card,1,18.12
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,3,voucher,1,2.00
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,2,voucher,1,18.59
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,1,boleto,1,141.46
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,1,credit_card,3,179.12


In [4]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

df['order_month'] = df['order_purchase_timestamp'].dt.to_period('M')

df_delivered = df[df['order_status'] == 'delivered'].copy()

print(f"Записей после фильтрации: {len(df_delivered)}")
df_delivered[['order_id', 'order_purchase_timestamp', 'order_month', 'payment_value']].head()

Записей после фильтрации: 100756


,order_id,order_purchase_timestamp,order_month,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10,18.12
1,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10,2.00
2,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10,18.59
3,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,2018-07,141.46
4,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,2018-08,179.12


In [9]:
total_sum_by_city = df_delivered.groupby('customer_city')['payment_value'].sum().reset_index()
top_10_cities = total_sum_by_city.sort_values(by='payment_value', ascending=False).head(10)
print(top_10_cities)

total_sum_by_month = df_delivered.groupby('order_month')['payment_value'].sum().reset_index()
total_sum_by_month = total_sum_by_month.sort_index()
print(total_sum_by_month)

       customer_city  payment_value
3563       sao paulo     2108162.22
3126  rio de janeiro     1111872.32
449   belo horizonte      406050.87
553         brasilia      345221.87
1135        curitiba      238582.26
2936    porto alegre      214861.83
700         campinas      209028.97
3218        salvador      207790.89
1518       guarulhos      157750.04
2440         niteroi      135458.19
   order_month  payment_value
0      2016-10       46566.71
1      2016-12          19.62
2      2017-01      127545.67
3      2017-02      271298.65
4      2017-03      414369.39
5      2017-04      390952.18
6      2017-05      567066.73
7      2017-06      490225.60
8      2017-07      566403.93
9      2017-08      646000.61
10     2017-09      701169.99
11     2017-10      751140.27
12     2017-11     1153528.05
13     2017-12      843199.17
14     2018-01     1078606.86
15     2018-02      966510.88
16     2018-03     1120678.00
17     2018-04     1132933.95
18     2018-05     1128836.69
19  

In [12]:
payment_analysis = df_delivered.groupby('payment_type')['payment_value'].agg(['count', 'mean'])
payment_analysis = payment_analysis.sort_values(by='count', ascending=False)
print(payment_analysis)

              count        mean
payment_type                   
credit_card   74586  162.243516
boleto        19191  144.334979
voucher        5493   62.445511
debit_card     1486  140.256474
